In [16]:
import pandas as pd
import urllib.request
import urllib.parse
from bs4 import BeautifulSoup

baseUrl = 'https://search.naver.com/search.naver?where=post&sm=tab_jum&query='
plusUrl = input("검색어를 입력하세요: ")
url = baseUrl+urllib.parse.quote_plus(plusUrl)

html = urllib.request.urlopen(url).read()
soup = BeautifulSoup(html, 'html.parser')

title = soup.find_all(class_='sh_blog_title')

searchList = []
for i in title:
    temp = []
    temp.append(i.attrs['title'])
    temp.append(i.attrs['href'])
    searchList.append(temp)

print(searchList)
df = pd.DataFrame(searchList, columns=['타이틀','주소'])
df.head()

검색어를 입력하세요: 파이썬
[['파이썬인강과 학원의 차이, 직접수강해본 후기 알려드립니다', 'https://blog.naver.com/cdh0584?Redirect=Log&logNo=221684600901'], ['파이썬 프로그래밍 - 개발 범위', 'https://blog.naver.com/urmyver?Redirect=Log&logNo=221670828962'], ['코딩기초 파이썬학원 직접 다녀본 후기입니다', 'https://blog.naver.com/designer_dh?Redirect=Log&logNo=221699076580'], ['일취월장 파이썬 학원 달라진 경험!', 'https://blog.naver.com/redrock27?Redirect=Log&logNo=221700209048'], ['한권으로 끝내는 파이썬 - 독학으로 Python 프로그램 코딩 공부하기', 'https://blog.naver.com/since201109?Redirect=Log&logNo=221553965961'], ['코랩(Colab)을 이용하여 파이썬 공부와 머신러닝 공부를 해요!~', 'https://chogar.blog.me/221665985078'], ['파이썬 아나콘다 설치하기 - Jupyter (쥬피터) 실행하기', 'https://blog.naver.com/roboholic84?Redirect=Log&logNo=221626686511'], ['[Python] 파이썬으로 와슨 AI 이용해보기', 'http://mcchae.egloos.com/11336272'], ['파이썬학원 기초 문법부터 심화까지 1달완성 교육기관!', 'https://blog.naver.com/ridesafe?Redirect=Log&logNo=221643768938'], ['라즈베리파이에서 파이썬으로 1602 I2C 캐릭터 LCD 사용하기', 'https://blog.naver.com/elepartsblog?Redirect=Log&logNo=221583231746']]


,타이틀,주소
0,"파이썬인강과 학원의 차이, 직접수강해본 후기 알려드립니다",https://blog.naver.com/cdh0584?Redirect=Log&lo...
1,파이썬 프로그래밍 - 개발 범위,https://blog.naver.com/urmyver?Redirect=Log&lo...
2,코딩기초 파이썬학원 직접 다녀본 후기입니다,https://blog.naver.com/designer_dh?Redirect=Lo...
3,일취월장 파이썬 학원 달라진 경험!,https://blog.naver.com/redrock27?Redirect=Log&...
4,한권으로 끝내는 파이썬 - 독학으로 Python 프로그램 코딩 공부하기,https://blog.naver.com/since201109?Redirect=Lo...


# 한진 배송페이지 sqlite3 (HanjinDelivery)

In [1]:
import pandas as pd
import urllib.request
import urllib.parse
from bs4 import BeautifulSoup
import http.client
import json
import numpy as np
import sqlite3
import csv

baseUrl = 'http://www.hanjin.co.kr/Delivery_html/inquiry/result_waybill.jsp?wbl_num='
plusUrl = input("송장번호를 입력하세요: ")            
url = baseUrl+urllib.parse.quote_plus(plusUrl)


df2= pd.read_html(url, encoding="CP949")[1]
del df2['Unnamed: 4']
# df2 = df2.drop(df2.index[-1], axis=0

df2.insert(loc=0, column='NUMBER', value=plusUrl)
df2['NUMBER'].astype('int64')
df2.to_csv("HanjinDeliveryDetail.csv", encoding="utf-8-sig",header=False, index=False, mode='a')



# 데이터베이스 연결
conn = sqlite3.connect('HanjinDelivery.db')

try:
  # 커서(Cursor) 생성
  # 커서: 명령문(SQL) 실행, 결과의 현재 위치 표시
  cursor = conn.cursor()
    
  # 테이블 생성(CREATE), 같은 이름의 테이블이 이미 있는 경우 제거(DROP)
  
  for i in df2.index:
      number =  df2.get_value(i, 'NUMBER')
      date = df2.get_value(i, '날짜')
      time = df2.get_value(i, '시간')
      location = df2.get_value(i, '상품위치')
      status = df2.get_value(i, '배송 진행상황')
      cursor.execute('INSERT INTO DeliveryDB VALUES (?, ?, ?, ?, ?)', (str(number), str(date), str(time), str(location), str(status)))

  # 트랜잭션 커밋 - 데이터베이스에 업데이트를 반영
  conn.commit()
  
  
  # 결과 확인
  # fetchall(): 쿼리의 결과를 모두 가져오기
  cursor.execute("SELECT * FROM DeliveryDB")
  result = cursor.fetchall()
  for r in result:
    print(r)
  
except sqlite3.Error as e:
  # 데이터베이스 에러 처리
  print("sqlite error: ", e)

finally:
  # 데이터베이스 연결 해제
  conn.close()

송장번호를 입력하세요: 400489097875
('0    400489097875\n1    400489097875\n2    400489097875\n3    400489097875\n4    400489097875\n5    400489097875\n6    400489097875\n7    400489097875\nName: NUMBER, dtype: object', '0    2019-10-18\n1    2019-10-18\n2    2019-10-19\n3    2019-10-19\n4    2019-10-19\n5    2019-10-21\n6    2019-10-21\n7    2019-10-21\nName: 날짜, dtype: object', '0    19:18\n1    21:53\n2    03:10\n3    08:16\n4    08:56\n5    09:37\n6    15:53\n7    15:53\nName: 시간, dtype: object', '0            중구터미널\n1            중구터미널\n2           남서울터미널\n3            김포터미널\n4            김포터미널\n5            김포(대)\n6            김포(대)\n7    수령인 : 김재*(기타)\nName: 상품위치, dtype: object', '0              고객님 상품을 집하하여 중구터미널에 입고되었습니다.\n1                  중구터미널에서 남서울터미널로 이동중 입니다.\n2                 남서울터미널에서 남서울터미널로 이동중 입니다.\n3                           김포터미널에 도착하였습니다.\n4                           배송원이 배송준비중 입니다.\n5    배송출발 하였습니다. (배송사원 : 김향빈 010-4609-7777)\n6    배송완료 되었습니다. (배송사원 : 김향빈 010-4609-7777)\

c:\users\yeongyun\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:37: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
c:\users\yeongyun\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:38: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
c:\users\yeongyun\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:39: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
c:\users\yeongyun\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:40: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
c:\users\yeongyun\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py

# 한진 배송페이지 MySQL (HanjinDelivery2)

In [3]:
pip install PyMySQL

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pymysql
import pandas as pd
import urllib.request
import urllib.parse
from bs4 import BeautifulSoup
import http.client
import json
import numpy as np
import csv


# MySQL Connection 연결
conn = pymysql.connect(host='localhost', user='root', password='mywprn0315!', db='test', charset='utf8')
    
baseUrl = 'http://www.hanjin.co.kr/Delivery_html/inquiry/result_waybill.jsp?wbl_num='
while True:
    plusUrl = input("송장번호를 입력하세요: ")            
    url = baseUrl+urllib.parse.quote_plus(plusUrl)


    df2= pd.read_html(url, encoding="CP949")[1]
    del df2['Unnamed: 4']
    # df2 = df2.drop(df2.index[-1], axis=0

    df2.insert(loc=0, column='NUMBER', value=plusUrl)
    df2['NUMBER'].astype('int64')
    df2.to_csv("HanjinDeliveryDetail2.csv", encoding="utf-8-sig",header=False, index=False, mode='a')



    # Connection 으로부터 Cursor 생성
    cursor = conn.cursor()

    # SQL문 실행

    for i in df2.index:
        number =  df2.get_value(i, 'NUMBER')
        date = df2.get_value(i, '날짜')
        time = df2.get_value(i, '시간')
        location = df2.get_value(i, '상품위치')
        status = df2.get_value(i, '배송 진행상황')
        cursor.execute("INSERT INTO DeliveryDB2 (number, date, time, location, status) VALUES (%s, %s, %s, %s, %s)", (number, date, time, location, status))


  # 트랜잭션 커밋 - 데이터베이스에 업데이트를 반영
    conn.commit()
  
  
    # 결과 확인
    # fetchall(): 쿼리의 결과를 모두 가져오기
    cursor.execute("SELECT * FROM DeliveryDB2")
    result = cursor.fetchall()
    for r in result:
        print(r)
    
    continue;
    
# 데이터베이스 연결 해제
conn.close()    

송장번호를 입력하세요: 414265558862


c:\users\yeongyun\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:37: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
c:\users\yeongyun\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:38: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
c:\users\yeongyun\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:39: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
c:\users\yeongyun\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:40: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
c:\users\yeongyun\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py

('414265558862', '2019-11-06', '16:43', '구리중앙(대)', '고객님 상품을 접수하였습니다.')
('414265558862', '2019-11-06', '17:11', '광진터미널', '고객님 상품을 집하하여 광진터미널에 입고되었습니다.')
('414265558862', '2019-11-06', '22:45', '광진터미널', '광진터미널에서 동서울터미널로 이동중 입니다.')
('414265558862', '2019-11-07', '02:09', '동서울터미널', '동서울터미널에 도착하였습니다.')
('414265558862', '2019-11-07', '05:01', '동서울터미널', '동서울터미널에서 양천터미널로 이동중 입니다.')
('414265558862', '2019-11-07', '06:29', '양천터미널', '양천터미널에 도착하였습니다.')
('414265558862', '2019-11-07', '09:26', '양천터미널', '배송원이 배송준비중 입니다.')
('414265558862', '2019-11-07', '12:56', '신화(대)', '배송출발 하였습니다. (배송사원 : 장병훈 010-9028-2243)')
('414265558862', '2019-11-07', '16:53', '신화(대)', '배송완료 되었습니다. (배송사원 : 장병훈 010-9028-2243)')
('414265558862', '2019-11-07', '16:53', '수령인 : 이소*(기타)', '수령인 : 이소*(기타)')
송장번호를 입력하세요: 400489097875
('414265558862', '2019-11-06', '16:43', '구리중앙(대)', '고객님 상품을 접수하였습니다.')
('414265558862', '2019-11-06', '17:11', '광진터미널', '고객님 상품을 집하하여 광진터미널에 입고되었습니다.')
('414265558862', '2019-11-06', '22:45', '광진터미널', '광진터